In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

games_df = pd.read_csv("games.csv")

# Из данных нужно исключить проекты ранее 2000 года и проекты, для которых имеются пропуски данных в любой из колонок.
filtered_games_df = games_df[games_df['Year_of_Release'] >= 2000].dropna()

filtered_games_df['User_Score'] = pd.to_numeric(filtered_games_df['User_Score'], errors='coerce').dropna()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Дашборд состояния игровой индустрии"),
    html.P("Этот дашборд дает представление об игровой индустрии. Отображены по очереди: 2 типа фильтров по жанрам и рейтингу, далее количество игр, 2 графика Stacked area plot, показывающий выпуск игр по годам и платформам и Scatter plot с разбивкой по жанрам. В конце фильтр по годам."),
    html.Div([
        html.Div([
            html.P("Фильтр по жанрам"),
            dcc.Checklist(
                id='genre-filter',
                options=[{'label': genre, 'value': genre} for genre in filtered_games_df['Genre'].unique()],
                value=filtered_games_df['Genre'].unique(),
                labelStyle={'display': 'block'}
            ),
        ], style={'width': '48%', 'display': 'inline-block'}),  # First column with 48% width
        
        html.Div([
            html.P("Фильтр по рейтингу"),
            dcc.Checklist(
                id='rating-filter',
                options=[{'label': rating, 'value': rating} for rating in filtered_games_df['Rating'].unique()],
                value=filtered_games_df['Rating'].unique(),
                labelStyle={'display': 'block'}
            ),
        ], style={'width': '48%', 'display': 'inline-block', 'marginLeft': '4%'}),  # Second column with 48% width and 4% left margin
    ]),
    
    html.P(id='num-games-text'),

    html.Div([
        html.Div([
            dcc.Graph(id='area-plot'),
        ], style={'width': '48%', 'display': 'inline-block'}),  # First column with 48% width
        
        html.Div([
            dcc.Graph(id='scatter-plot'),
        ], style={'width': '48%', 'display': 'inline-block', 'marginLeft': '4%'}),  # Second column with 48% width and 4% left margin
    ]),
    
    html.P("Фильтр по годам"),
    dcc.RangeSlider(
        id='year-filter',
        min=filtered_games_df['Year_of_Release'].min(),
        max=filtered_games_df['Year_of_Release'].max(),
        value=[filtered_games_df['Year_of_Release'].min(), filtered_games_df['Year_of_Release'].max()],
        marks={year: str(int(year)) for year in filtered_games_df['Year_of_Release'].unique()}
    )
])

@app.callback(
    [Output('num-games-text', 'children'),
     Output('area-plot', 'figure'),
     Output('scatter-plot', 'figure')],
    [Input('genre-filter', 'value'),
     Input('rating-filter', 'value'),
     Input('year-filter', 'value')]
)
def update_dashboard(selected_genres, selected_ratings, selected_years):
    df = filtered_games_df[
        (filtered_games_df['Genre'].isin(selected_genres)) & 
        (filtered_games_df['Rating'].isin(selected_ratings)) & 
        (filtered_games_df['Year_of_Release'] >= selected_years[0]) & 
        (filtered_games_df['Year_of_Release'] <= selected_years[1])
    ]
    
    num_games = len(df)
    num_games_text = f"Количество выбранных игр: {num_games}"
    
    area_df = df.groupby(['Year_of_Release', 'Platform']).size().reset_index(name='count')
    area_fig = px.area(area_df, x='Year_of_Release', y='count', color='Platform', title='Выпуск игр по годам и платформам')
    
    scatter_fig = px.scatter(df, x='User_Score', y='Critic_Score', color='Genre', title='Оценки критиков vs пользователей по жанру')
    
    return num_games_text, area_fig, scatter_fig

if __name__ == '__main__':
    app.run(debug=True)


/var/folders/x0/bzrtlp_150sb29kprmm15lkr0000gn/T/ipykernel_39228/3057386775.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/x0/bzrtlp_150sb29kprmm15lkr0000gn/T/ipykernel_39228/3057386775.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [17]:
games_df['Rating'].unique()

,Name,Platform,Year_of_Release,Genre,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,NaN,NaN,NaN
